In [ ]:
# Import needed libraries
from urllib import request
import shutil
import ssl
import zipfile
import xarray
import netCDF4 as nc
import numpy as np
import os
ssl._create_default_https_context = ssl._create_unverified_context

# Sets up arrays to download all data
RFCArray=['ABRFC', 'CBRFC','CNRFC', 'LMRFC', 'MARFC', 'MBRFC', 'NCRFC', 'NERFC', 'NWRFC', 'OHRFC', 'SERFC', 'WGRFC'];
YearArray=list(range(1979,1981));
print(YearArray)
MonthArray=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'];
MonthDays=[31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 30];
            
DayArray31=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12','13', '14', '15', '16', '17', '18', '19', '20', '21', '22','23','24','25','26','27','28','29','30','31'];
DayArray28=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12','13', '14', '15', '16', '17', '18', '19', '20', '21', '22','23','24','25','26','27','28'];
DayArray29=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12','13', '14', '15', '16', '17', '18', '19', '20', '21', '22','23','24','25','26','27','28','29'];
DayArray30=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12','13', '14', '15', '16', '17', '18', '19', '20', '21', '22','23','24','25','26','27','28','29','30'];
HourArray=['00','01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12','13', '14', '15', '16', '17', '18', '19', '20', '21', '22','23'];


In [ ]:
for aa in YearArray:
    Year=str(aa);
    for bb in MonthArray:
        Month=bb;
        if Year== '1979' and Month=='01':
            continue     
        for cc in RFCArray:
            RFC=cc;
            URL = "https://hydrology.nws.noaa.gov/aorc-historic/AORC_"+RFC+ "_4km/"+RFC+"_precip_partition/AORC_APCP_4KM_"+RFC+"_"+Year+Month+".zip";
            FileName="AORC_APCP_4KM_"+RFC+"_"+Year+Month+".zip";
            response = request.urlretrieve(URL, "Zips/"+FileName)
            with zipfile.ZipFile("Zips/"+FileName, 'r') as zip_ref:
                zip_ref.extractall("Data")
        
        MonthNum=int(Month)-1;
        MonthSize=MonthDays[MonthNum]
        
        if MonthSize==31:
            DayArray=DayArray31;
            
        if MonthSize==30:
            DayArray=DayArray30;
            
        if MonthSize==28:
            DayArray=DayArray28;
            
        if (int(aa) % 4 ==0):
            DayArray=DayArray29;
                
        for cc in DayArray:
            Day=cc
            for dd in HourArray:
                Hour=dd;
                d1=xarray.open_dataset('Data/AORC_APCP_ABRFC_'+Year+Month+Day+Hour+'.nc4')
                for ee in RFCArray:
                    if ee=='ABRFC':
                        continue
                        
                    d2=xarray.open_dataset('Data/AORC_APCP_'+ee+'_'+Year+Month+Day+Hour+'.nc4')
                    ds= xarray.merge([d1,d2])
                    d1=ds
                       
                ds.to_netcdf('Finished/AORC_APCP_'+Year+Month+Day+Hour+'.nc')
